In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Rectal_Cancer"
cohort = "GSE150082"

# Input paths
in_trait_dir = "../../input/GEO/Rectal_Cancer"
in_cohort_dir = "../../input/GEO/Rectal_Cancer/GSE150082"

# Output paths
out_data_file = "../../output/preprocess/Rectal_Cancer/GSE150082.csv"
out_gene_data_file = "../../output/preprocess/Rectal_Cancer/gene_data/GSE150082.csv"
out_clinical_data_file = "../../output/preprocess/Rectal_Cancer/clinical_data/GSE150082.csv"
json_path = "../../output/preprocess/Rectal_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Pre-existing tumoral B cell infiltration and impaired genome maintenance correlate with response to chemoradiotherapy in locally advanced rectal cancer (LARC)"
!Series_summary	"Using Human Genome 4x44 two-color Agilent microarrays, we established the expression profiling of 39 LARC pretreatment tumor samples to elucidate the molecular features associated with response to treatment after neoadjuvant chemoradiotherapy (nCRT)."
!Series_overall_design	"Two color microarrays where Cy5= tumor sample and Cy3= Stratagene Universal Human RNA Reference. This dataset comprises the transcriptomic profiling of 39 consecutive eligible LARC patients who underwent therapy at the Oncology Unit at Bonorino Udaondo Hospital (Buenos Aires, Argentina) from November 2015 to September 2018. This study was approved by the Udaondo Hospital Ethics Committee and the Instituto Leloir Institutional Review Board. All patients signed the approved Informed Consent.  All patients

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analyze gene expression data availability
is_gene_available = True  # Based on Series_summary mentioning "expression profiling" and "Agilent microarrays"

# Define trait row and conversion function
trait_row = 4  # 'response' field contains binary Good/Poor response data
age_row = 2    # Age data is available
gender_row = 0  # Sex data is available

# Define conversion functions for clinical features
def convert_trait(value):
    """Convert trait value to binary: Good response = 1, Poor response = 0"""
    if value is None:
        return None
    
    value_part = value.split(': ')[-1].strip() if ': ' in value else value.strip()
    
    if value_part == 'Good':
        return 1
    elif value_part == 'Poor':
        return 0
    else:
        return None

def convert_age(value):
    """Convert age value to continuous numeric value"""
    if value is None:
        return None
    
    value_part = value.split(': ')[-1].strip() if ': ' in value else value.strip()
    
    try:
        return float(value_part)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender value to binary: Female = 0, Male = 1"""
    if value is None:
        return None
    
    value_part = value.split(': ')[-1].strip() if ': ' in value else value.strip()
    
    if value_part == 'F':
        return 0
    elif value_part == 'M':
        return 1
    else:
        return None

# Determine trait availability
is_trait_available = trait_row is not None

# Save metadata for initial filtering
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# Extract clinical features if trait data is available
if trait_row is not None:
    # Assuming clinical_data is already in memory from previous steps
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted data
    preview_data = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview_data)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the extracted clinical data
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM4523129': [1.0, 70.0, 1.0], 'GSM4523130': [1.0, 74.0, 1.0], 'GSM4523131': [1.0, 45.0, 0.0], 'GSM4523132': [1.0, 45.0, 0.0], 'GSM4523133': [1.0, 54.0, 1.0], 'GSM4523134': [1.0, 72.0, 1.0], 'GSM4523135': [1.0, 57.0, 1.0], 'GSM4523136': [1.0, 66.0, 1.0], 'GSM4523137': [1.0, 71.0, 0.0], 'GSM4523138': [1.0, 47.0, 1.0], 'GSM4523139': [1.0, 66.0, 1.0], 'GSM4523140': [1.0, 61.0, 1.0], 'GSM4523141': [1.0, 64.0, 1.0], 'GSM4523142': [1.0, 61.0, 1.0], 'GSM4523143': [1.0, 59.0, 1.0], 'GSM4523144': [0.0, 34.0, 0.0], 'GSM4523145': [0.0, 63.0, 1.0], 'GSM4523146': [0.0, 66.0, 0.0], 'GSM4523147': [0.0, 46.0, 1.0], 'GSM4523148': [0.0, 57.0, 1.0], 'GSM4523149': [0.0, 64.0, 1.0], 'GSM4523150': [0.0, 55.0, 1.0], 'GSM4523151': [0.0, 75.0, 1.0], 'GSM4523152': [0.0, 61.0, 1.0], 'GSM4523153': [0.0, 42.0, 0.0], 'GSM4523154': [0.0, 63.0, 1.0], 'GSM4523155': [0.0, 42.0, 1.0], 'GSM4523156': [0.0, 69.0, 1.0], 'GSM4523157': [0.0, 49.0, 0.0], 'GSM4523158': [0.0, 68.0, 0.0], 

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print(gene_data.index[:20])


Index(['A_23_P100001', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074',
       'A_23_P100127', 'A_23_P100141', 'A_23_P100189', 'A_23_P100196',
       'A_23_P100203', 'A_23_P100220', 'A_23_P100240', 'A_23_P10025',
       'A_23_P100292', 'A_23_P100315', 'A_23_P100326', 'A_23_P100344',
       'A_23_P100355', 'A_23_P100386', 'A_23_P100392', 'A_23_P100420'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Reviewing the gene identifiers in the dataset
# These identifiers (A_23_P100001, etc.) are Agilent microarray probe IDs
# They are not human gene symbols and will need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107', '(+)E1A_r60_a135'], 'SPOT_ID': ['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107', '(+)E1A_r60_a135'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'REFSEQ': [nan, nan, nan, nan, nan], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'CYTOBAND': [nan, nan, nan, nan, nan], 'DESCRIPTION': [nan, nan, nan, nan, nan], 'GO_ID': [nan, nan, nan, nan, nan], 'SEQUENCE': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# Looking at the gene identifiers in gene_data (from step 3) like 'A_23_P100001'
# and the gene annotation preview (from step 5)
# We need to find the appropriate columns for mapping

# First examine the annotation columns that potentially map to gene identifiers in gene_data
# Let's print more rows to find example of probe IDs that match our gene_data format
print("Looking for probe IDs matching the format in gene_data:")
for i in range(20):
    if i < len(gene_annotation):
        if str(gene_annotation['ID'].iloc[i]).startswith('A_23_P'):
            print(f"Found matching probe ID at row {i}: {gene_annotation['ID'].iloc[i]}")
            break

# Get a sample to see what column contains the gene symbols
sample_rows = gene_annotation[gene_annotation['ID'].str.startswith('A_23_P', na=False)].head(5)
print("\nSample rows with matching probe IDs:")
print(sample_rows[['ID', 'GENE_SYMBOL']])

# Now create the mapping dataframe using the identified columns
# The 'ID' column in gene_annotation corresponds to probe IDs in gene_data
# The 'GENE_SYMBOL' column contains the gene symbols
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')
print("\nPreview of mapping dataframe:")
print(mapping_df.head())

# Apply the gene mapping to convert probe-level measurements to gene expression
gene_expression_data = apply_gene_mapping(gene_data, mapping_df)

# Normalize gene symbols to ensure consistency
gene_data = normalize_gene_symbols_in_index(gene_expression_data)

# Preview the gene expression data
print("\nPreview of gene expression data after mapping:")
print(gene_data.head())

# Save the processed gene data to the output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


Looking for probe IDs matching the format in gene_data:
Found matching probe ID at row 11: A_23_P100001



Sample rows with matching probe IDs:
              ID GENE_SYMBOL
11  A_23_P100001     FAM174B
12  A_23_P100022        SV2B
13  A_23_P100056      RBPMS2
14  A_23_P100074        AVEN
15  A_23_P100127       CASC5



Preview of mapping dataframe:
              ID     Gene
11  A_23_P100001  FAM174B
12  A_23_P100022     SV2B
13  A_23_P100056   RBPMS2
14  A_23_P100074     AVEN
15  A_23_P100127    CASC5



Preview of gene expression data after mapping:
          GSM4523129  GSM4523130  GSM4523131  GSM4523132  GSM4523133  \
Gene                                                                   
A1BG       -4.423492   -3.130753   -3.654191   -3.428902   -3.588846   
A1BG-AS1   -3.023192   -1.816686   -1.816458   -2.099744   -2.114976   
A1CF        1.322759   -1.244949    0.749600    2.108298    1.239829   
A2M        -2.857169   -2.293804   -2.676066   -2.405703   -0.954434   
A2ML1       0.249256   -0.328081   -0.652739   -0.052724   -0.723140   

          GSM4523134  GSM4523135  GSM4523136  GSM4523137  GSM4523138  ...  \
Gene                                                                  ...   
A1BG       -3.866785   -3.539964   -3.925691   -1.980177   -4.693128  ...   
A1BG-AS1   -2.179731   -1.799732   -2.380574   -1.746726   -2.501482  ...   
A1CF        1.464439    2.489900    1.403291   -1.993101    1.871084  ...   
A2M        -2.226460   -2.769710   -1.788050   -1.903759   -2.


Gene expression data saved to ../../output/preprocess/Rectal_Cancer/gene_data/GSE150082.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Extract clinical features
clinical_features = geo_select_clinical_features(
    clinical_data, 
    trait=trait, 
    trait_row=trait_row, 
    convert_trait=convert_trait,
    age_row=age_row, 
    convert_age=convert_age,
    gender_row=gender_row, 
    convert_gender=convert_gender
)

# Save the clinical features data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print(linked_data.head())

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains gene expression data from CD4 T-cells of pSS patients and healthy controls."
)

# 6. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")

Clinical data saved to ../../output/preprocess/Rectal_Cancer/clinical_data/GSE150082.csv
Normalized gene data shape: (19447, 39)
First few normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS']


Normalized gene data saved to ../../output/preprocess/Rectal_Cancer/gene_data/GSE150082.csv
Linked data shape: (39, 19450)
            Rectal_Cancer   Age  Gender      A1BG  A1BG-AS1      A1CF  \
GSM4523129            1.0  70.0     1.0 -4.423492 -3.023192  1.322759   
GSM4523130            1.0  74.0     1.0 -3.130753 -1.816686 -1.244949   
GSM4523131            1.0  45.0     0.0 -3.654191 -1.816458  0.749600   
GSM4523132            1.0  45.0     0.0 -3.428902 -2.099744  2.108298   
GSM4523133            1.0  54.0     1.0 -3.588846 -2.114976  1.239829   

                 A2M     A2ML1    A4GALT     A4GNT  ...    ZWILCH     ZWINT  \
GSM4523129 -2.857169  0.249256 -0.808312  0.532630  ... -2.439972 -1.899276   
GSM4523130 -2.293804 -0.328081 -1.429592  0.752957  ... -2.274822 -3.385446   
GSM4523131 -2.676066 -0.652739 -1.259287  0.354724  ... -2.184009 -1.220591   
GSM4523132 -2.405703 -0.052724 -0.942970  0.561949  ... -2.805278 -3.204676   
GSM4523133 -0.954434 -0.723140 -0.490592  0

Shape after handling missing values: (39, 19450)
For the feature 'Rectal_Cancer', the least common label is '1.0' with 16 occurrences. This represents 41.03% of the dataset.
The distribution of the feature 'Rectal_Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 51.5
  50% (Median): 61.0
  75%: 66.0
Min: 30.0
Max: 75.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 10 occurrences. This represents 25.64% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Rectal_Cancer/GSE150082.csv
